# Импорт библиотек и данных

In [4]:
import sys
from pathlib import Path

import pandas as pd
import numpy as np

import mlflow
from mlflow.models import infer_signature

from sklearn.linear_model import LinearRegression

root_folder = '../'
sys.path.append(root_folder)

baseline_data = Path(root_folder, 'data', 'processed', '1.0_baseline.csv')

In [5]:
df = pd.read_csv(
    baseline_data,
    index_col=0
)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 374306 entries, 0 to 374305
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   baths         267467 non-null  float64
 1   fireplace     374306 non-null  bool   
 2   sqft          333384 non-null  float64
 3   beds          277504 non-null  float64
 4   stories       221109 non-null  float64
 5   target        374306 non-null  int64  
 6   private_pool  374306 non-null  bool   
 7   mls           374306 non-null  bool   
 8   year_built    311041 non-null  float64
dtypes: bool(3), float64(5), int64(1)
memory usage: 21.1 MB
